In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


        
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/train.csv')

X = train.iloc[:, 1:94]
y = train[['target']]

print(train.groupby('target')['id'].count())

y = pd.get_dummies(y, columns=['target'])



X_train, X_dev_test, y_train, y_dev_test = train_test_split(X, y, test_size=0.2)
X_dev, X_test, y_dev, y_test = train_test_split(X_dev_test, y_dev_test, test_size=0.5)

print('X_train shape: %s' % str(X_train.shape))
print('y_train shape: %s' % str(y_train.shape))
print('X_dev shape: %s' % str(X_dev.shape))
print('y_dev shape: %s' % str(y_dev.shape))
print('X_test shape: %s' % str(X_test.shape))
print('y_test shape: %s' % str(y_test.shape))








In [ ]:
# fit logistic regression model using keras
model = tf.keras.models.Sequential(
    [
        
      tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),

      tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
  
      tf.keras.layers.Dense(9, activation='softmax'),
    ]
)



model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-3), metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_dev, y_dev), batch_size=64)

loss = history.history['loss']
accuracy = history.history['accuracy']
val_loss = history.history['val_loss']
val_accuracy = history.history['val_accuracy']

iteration = range(1, len(history.history['loss'])+1)

# visualize loss
loss_df = pd.DataFrame({'loss': loss, 'val_loss': val_loss}, index=iteration)
loss_df.plot.line()

# visualize accuracy
acc_df = pd.DataFrame({'accuracy': accuracy, 'val_accuracy': val_accuracy}, index=iteration)
acc_df.plot.line()




In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
submit = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/test.csv')
X_submit = submit.drop(columns=['id'])
#predictions = np.greater(model.predict(X_submit), 0.5).astype(int)
predictions = model.predict(X_submit)
pred = pd.DataFrame(predictions, columns=['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9'])

result = pd.concat([submit[['id']], pred], axis=1)

result.to_csv('/kaggle/working/output.csv', index=False)
print(result.head())

#sample = pd.read_csv('/kaggle/input/otto-group-product-classification-challenge/sampleSubmission.csv')
#print(sample.head())